In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [2]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [3]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [4]:
print(fountains.crs)

None


In [5]:
fountains = fountains.set_crs('epsg:4326')

In [6]:
print(fountains.crs)

epsg:4326


In [7]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [8]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Bike path

In [9]:
bikepath = gpd.read_file(os.getenv("bike_path"))
bikepath.columns = bikepath.columns.str.lower()

In [10]:
print(bikepath.crs)

EPSG:4326


In [11]:
columns = ["id_cycl", "nom_arr_ville_r", "geometry"]
bikepath = bikepath.filter(columns)

### calcul the colsest distance between fountain and bike path

In [12]:
fountains_utm = fountains.to_crs(epsg=32188)
bikepath_utm = bikepath.to_crs(epsg=32188)

In [13]:
print(fountains_utm.crs)
print(bikepath_utm.crs)

EPSG:32188
EPSG:32188


In [14]:
fountain_w_bikepath_data = fountains_utm.sjoin_nearest(bikepath_utm, distance_col="nearest_bikepath")

In [15]:
min(fountain_w_bikepath_data["nearest_bikepath"])

0.6113412388749414

In [16]:
max(fountain_w_bikepath_data["nearest_bikepath"])

815.7034137786129

In [17]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [18]:
part = fountain_w_bikepath_data[fountain_w_bikepath_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [19]:
m

In [20]:
part = bikepath[bikepath["nom_arr_ville_r"]=="Ville-Marie"]
folium.Choropleth(part.geometry, line_weight=2, line_color="blue").add_to(m)

In [ ]:
m

In [38]:
part = fountain_w_bikepath_data[fountain_w_bikepath_data.nom_parc_lieu.str.contains("Royal", na=False)]

In [44]:
print(f"min: {min(part.nearest_bikepath)}")
print(f"max: {max(part.nearest_bikepath)}")

min: 1.687542633888428
max: 127.00956498909619


In [40]:
part

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry,index_right,id_cycl,nom_arr_ville_r,nearest_bikepath
829,843,Ville-Marie,du Mont-Royal,sentier,NaN,Grands Parcs,-73.587095,45.502097,POINT (297993.790 5040245.380),268,27178.0,Ville-Marie,5.809520
846,860,Ville-Marie,du Mont-Royal,sentier,NaN,Grands Parcs / auge pour chevaux,-73.587196,45.502090,POINT (297985.900 5040244.650),268,27178.0,Ville-Marie,9.277082
620,631,Ville-Marie,près du parc du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,au pied de l'escalier,-73.582003,45.504592,POINT (298391.930 5040522.260),2517,27177.0,Ville-Marie,68.039099
841,855,Ville-Marie,du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,Grands Parcs,-73.583063,45.505103,POINT (298309.170 5040579.060),2517,27177.0,Ville-Marie,5.760412
827,841,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587419,45.503420,POINT (297968.620 5040392.460),456,27522.0,Ville-Marie,69.605840
828,842,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.586842,45.503720,POINT (298013.690 5040425.750),456,27522.0,Ville-Marie,67.645584
832,846,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.588005,45.504188,POINT (297922.910 5040477.850),456,27522.0,Ville-Marie,25.725551
830,844,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.591708,45.502730,POINT (297633.330 5040316.150),217,27172.0,Ville-Marie,3.968114
831,845,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587162,45.504390,POINT (297988.780 5040500.190),2526,27523.0,Ville-Marie,8.095345
833,847,Ville-Marie,du Mont-Royal,chalet,près du lac des castors,Grands Parcs,-73.599150,45.498618,POINT (297051.250 5039859.870),644,28527.0,Ville-Marie,87.217950
